# AI safety via debate
OAI article: https://openai.com/index/debate/

Paper: https://arxiv.org/pdf/1805.00899

Implementation by Alex Chen

Jul 2024

## Example Output

Here, the judge was fooled by the liar agent (red), despite the truth agent's (blue) best efforts.

![AI Safety Debate](OAISafetyDebateImg.png)


# Setup

In [3]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from torchvision.transforms import ToTensor
import math

# Training the Judge
The judge is a classifier trained to "predict MNIST digits from 6 non-black digits, sampled at random for each presentation when pretraining."

As per the paper, their original sparse classifier achieved 59.4% accuracy. They use a convolutional neural net.

In [4]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.0005
momentum = 0.5
log_interval = 10

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision.transforms import ToTensor

class RandomPixelSelection(object):
    def __init__(self, num_pixels=6):
        self.num_pixels = num_pixels

    def __call__(self, img):
        img_np = np.array(img)
        flattened = img_np.flatten()
        non_black_indices = np.where(flattened > 0)[0]

        if len(non_black_indices) < self.num_pixels:
            selected_indices = non_black_indices
        else:
            selected_indices = np.random.choice(non_black_indices, self.num_pixels, replace=False)

        new_img = np.zeros_like(flattened)
        new_img[selected_indices] = flattened[selected_indices]
        new_img = new_img.reshape(img_np.shape)

        new_img = torch.tensor(new_img, dtype=torch.float32)
        new_img = (new_img - 0.1307) / 0.3081

        return new_img.unsqueeze(0)

transform = torchvision.transforms.Compose([
    RandomPixelSelection(num_pixels=6)
])

batch_size_train = 64
batch_size_test = 1000

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('~/mnist_data/', train=True, download=True, transform=transform),
    batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('~/mnist_data/', train=False, download=True, transform=transform),
    batch_size=batch_size_test, shuffle=True)

class Judge(nn.Module):
    def __init__(self):
        super(Judge, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
        self.fc1 = nn.Linear(12 * 20 * 20, 60)
        self.fc2 = nn.Linear(60, 30)
        self.fc3 = nn.Linear(30, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 12 * 20 * 20)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Judge().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

n_epochs = 8
for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.696778
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.022275
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.931807
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.689541
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.629298
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.643373
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.779793
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.563575
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.616541
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.785779

Test set: Average loss: 1.5354, Accuracy: 4616/10000 (46%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.699187
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.482269
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.422606
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.560940
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.653389
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.583228
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.364765
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.646239
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.264989
T

In [6]:
example_data, example_target = next(iter(test_loader))
example_data, example_target = example_data.to(device), example_target.to(device)
model.eval()
with torch.no_grad():
    output = model(example_data)
    pred = output.argmax(dim=1, keepdim=True)
    for i in range(len(pred)):
        print(f'Predicted: {pred[i].item()}, Actual: {example_target[i].item()}')


Predicted: 7, Actual: 9
Predicted: 1, Actual: 1
Predicted: 7, Actual: 5
Predicted: 7, Actual: 6
Predicted: 1, Actual: 1
Predicted: 6, Actual: 6
Predicted: 4, Actual: 7
Predicted: 6, Actual: 6
Predicted: 0, Actual: 7
Predicted: 1, Actual: 1
Predicted: 8, Actual: 9
Predicted: 7, Actual: 9
Predicted: 6, Actual: 8
Predicted: 2, Actual: 8
Predicted: 6, Actual: 6
Predicted: 6, Actual: 5
Predicted: 8, Actual: 8
Predicted: 2, Actual: 2
Predicted: 1, Actual: 1
Predicted: 7, Actual: 7
Predicted: 0, Actual: 6
Predicted: 0, Actual: 6
Predicted: 7, Actual: 7
Predicted: 9, Actual: 9
Predicted: 4, Actual: 5
Predicted: 2, Actual: 2
Predicted: 5, Actual: 5
Predicted: 8, Actual: 9
Predicted: 2, Actual: 8
Predicted: 7, Actual: 0
Predicted: 5, Actual: 5
Predicted: 0, Actual: 0
Predicted: 5, Actual: 5
Predicted: 7, Actual: 0
Predicted: 0, Actual: 5
Predicted: 9, Actual: 8
Predicted: 8, Actual: 8
Predicted: 6, Actual: 0
Predicted: 3, Actual: 5
Predicted: 8, Actual: 2
Predicted: 7, Actual: 7
Predicted: 1, Ac

In [7]:
class StandardMNIST(object):
    def __call__(self, img):
        img_np = np.array(img)
        new_img = torch.tensor(img_np, dtype=torch.float32)
        return new_img.unsqueeze(0)

standardTransform = torchvision.transforms.Compose([
    StandardMNIST(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [8]:
batch_size_test = 1

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('~/mnist_data/', train=False, download=True, transform=standardTransform),
    batch_size=batch_size_test, shuffle=True)

In [9]:
example_image, example_label = next(iter(test_loader))
example_image_dim = example_image[0][0]
print(example_label)

tensor([8])


# The Agent

The selection criteria is the PUCT variant: the win rate + Upper bound

the root of the tree is the empty state

first move is truthful agent